## Text UniSim Demo

This demo showcases how to use Text UniSim (TextSim) for efficient fuzzy string matching, near-duplicate detection, and string similarity using a real-world entity matching dataset.

For additional information, please see the documentation on [GitHub](https://github.com/google/unisim). For more details on the RETSim model used by UniSim, please see the [TODO RETSim paper]().

In [1]:
# installing needed dependencies
try:
    import unisim
except ImportError:
    !pip install unisim

try:
    import datasets
except ImportError:
    !pip install datasets

Loaded backend
Using tf with gpu


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# imports
from datasets import load_dataset
from tabulate import tabulate
import pandas as pd

In [3]:
# import TextSim from UniSim
from unisim import TextSim

## Load Dataset

For this demo, we use entity matching datasets available on [Huggingface](https://huggingface.co/datasets/RUC-DataLab/ER-dataset). We use the `restaurants1.csv` dataset which contains restaurants' names/phone numbers/addresses for this colab.

Feel free to explore other examples they offer such as product matching (`walmart_amazon.csv`), paper citation matching (`dblp_scholar.csv`), and beer brands (`beer.csv`). The public datasets are from [DeepMatcher](https://github.com/anhaidgroup/deepmatcher/blob/master/Datasets.md), [Magellan](https://sites.google.com/site/anhaidgroup/useful-stuff/the-magellan-data-repository) and [WDC](http://webdatacommons.org/largescaleproductcorpus/v2/) and you can find a summary of them [here](https://github.com/ruc-datalab/DADER/tree/main).

In [4]:
# load dataset from huggingface
ds_name = "restaurants1.csv"
dataset = load_dataset("RUC-DataLab/ER-dataset", data_files=ds_name, split="train")

print("Size of dataset:", len(dataset))

dataset_features = list(dataset.features.keys())
print("Dataset features:", dataset_features)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Size of dataset: 450
Dataset features: ['A_NAME', 'A_PHONENUMBER', 'A_ADDRESS', 'B_NAME', 'B_PHONENUMBER', 'B_ADDRESS', 'label']


In [5]:
# get features corresponding to pairs of texts in the dataset (text1, text2)
text1_features = [x for x in dataset_features if x.startswith("A")]
text2_features = [x for x in dataset_features if x.startswith("B")]
is_match_feature = "label"

# create text pairs for example at idx in the dataset
def get_text_pair(idx):
    ex = dataset[idx]

    text1 = " ".join(str(ex[x]) for x in text1_features)
    text2 = " ".join(str(ex[x]) for x in text2_features)

    label = ex[is_match_feature]
    return [text1, text2, label]

#### Initialize TextSim

TextSim supports GPU acceleration (set `use_accelerator=True`) with a specified `batch_size` parameter. If a GPU is not detected, TextSim will default to CPU. By default, TextSim saves a copy of your dataset but you can set `store_data=False` to save memory when using larger datasets.

Additionally, TextSim support Approximate Nearest Neighbor (ANN) search through [USearch](https://github.com/unum-cloud/usearch). Setting `index_type="approx"` will make TextSim significantly faster on large datasets (sub-linear search time). However, please note that while ANN search is very accurate, it does not guarantee that it will always find the closest match to a search query.

In [6]:
# create TextSim using default parameter settings
text_sim = TextSim(
    store_data=True, # set to False for large datasets to save memory
    index_type="exact", # set to "approx" for large datasets to use ANN search
    batch_size=128, # increasing batch_size on GPU may be faster
    use_accelerator=True, # uses GPU if available, otherwise uses CPU
)

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


INFO: UniSim is storing a copy of the indexed data
INFO: If you are using large data corpus, consider disabling this behavior using store_data=False


#### Computing Similarity between Strings

You can compute the similarity between two strings using the `.similarity(string1, string2)` method. The similarity value is a float between 0 and 1, with 1.0 representing identical strings. This is the cosine similarity between the vector representations of the strings.

In this example, we compute the similarity between the first 5 pairs of the dataset.

In [7]:
example_data = [get_text_pair(idx) for idx in range(0, 5)]

for i in range(len(example_data)):
    text1, text2, is_match = example_data[i]  # ground truth is_match label

    # compute similarity between text pair using .similarity
    similarity = text_sim.similarity(text1, text2)

    example_data[i].append(similarity)

# display results in df
df = pd.DataFrame(example_data, columns=["text1", "text2", "match_label", "similarity"])
display(df.head())

,text1,text2,match_label,similarity
0,"15 Romolo (415) 398-1359 15 Romolo Place, San ...","15 Romolo (415) 398-1359 15 Romolo Pl, San Fra...",1,0.971069
1,"456 Shanghai Cuisine 1261 69 Mott Street, New ...",Shanghai Asian Manor (212) 766-6311 21 Mott St...,0,0.779613
2,5A5 Steak Lounge (415) 989-2539 244 Jackson St...,Delicious Dim Sum (415) 781-0721 752 Jackson S...,0,0.687863
3,"9th Street Pizza (213) 627-7798 231 E 9th St, ...",Han Bat Sul Lung Tang (213) 383-9499 4163 W 5t...,0,0.610497
4,"9th Street Pizza (213) 627-7798 231 E 9th St, ...",Jun Won Restaurant (213) 383-8855 3100 W 8th S...,0,0.617363


We can see that the similarity between the first pair is 0.97, which is very high and indicates that the strings are near-duplicates of each other and the addresses match. The other pairs have far lower similarity values, indicating they are likely not matching strings and indeed, they do not represent the same entity.

#### Fuzzy String Matching

TextSim offers efficient fuzzy string matching between two lists using the `.match` function. The `.match` function accepts `queries` (list of strings you want to find matches for) and `targets` (list of strings you are finding matches in). It returns a Pandas DataFrame, where each row contains a query, its most similar match found in targets, their similarity, and whether or not they are a match (if their similarity is >= `similarity_threshold`). `0.9` is a good starting point for `similarity_threshold` when matching near-duplicate strings.

In this example, we show that TextSim is able to match restaurant addresses accurately even when there are typos, abbreviations, and formatting differences.

In [8]:
# targets to match queries to
targets = [
    "Shanghai Asian Manor (212) 766-6311 21 Mott St, New York, NY 10013",
    "Delicious Dim Sum (415) 781-0721 752 Jackson St, San Francisco, CA 94133",
    "15 Romolo (415) 398-1359 15 Romolo Pl, San Francisco, CA 94133",
]

# search queries we are looking up and finding matches for
queries = [
    "Shanghai asia manor (212)-766-6311 21 Mott street, New York, NY 94133", # near-dup match (capitalization, typos, different format)
    "Googleplex (650) 253-0000 1600 Amphitheatre Pkwy, Mountain View, CA 94043", # no match
    "Sino-american books & arts (415) 421-3345 751 Jackson St, San Francisco, CA 94133", # no match, different places but similar address
]

# .match does fuzzy matching between queries and targets lists
results_df = text_sim.match(queries, targets, similarity_threshold=0.9)

# display results dataframe
with pd.option_context('display.max_colwidth', None):
    display(results_df.head(10))

,query,target,similarity,is_match
0,"Shanghai asia manor (212)-766-6311 21 Mott street, New York, NY 94133","Shanghai Asian Manor (212) 766-6311 21 Mott St, New York, NY 10013",0.906984,True
1,"Googleplex (650) 253-0000 1600 Amphitheatre Pkwy, Mountain View, CA 94043","15 Romolo (415) 398-1359 15 Romolo Pl, San Francisco, CA 94133",0.466477,False
2,"Sino-american books & arts (415) 421-3345 751 Jackson St, San Francisco, CA 94133","Delicious Dim Sum (415) 781-0721 752 Jackson St, San Francisco, CA 94133",0.746303,False


We can try this on the whole dataset now. We use the first text in each pair as the target and the second text as the search query.

In [9]:
targets = list(set([get_text_pair(idx)[0] for idx in range(0, len(dataset))]))
queries = list(set([get_text_pair(idx)[1] for idx in range(0, len(dataset))]))

print("Dataset examples:")
print("\n".join([t for t in targets[:5]]))

Dataset examples:
Spur Tree Lounge (212) 477-9977 74 Orchard Street, New York, NY
Sushi Boat (415) 781-5111 389 Geary Street, San Francisco, CA
Falling Water Cafe (608) 831-2000 1313 John Q Hammons Dr, Middleton, WI
Shallots Bistro (847) 677-3463 7016 Carpenter Road, Skokie, IL
Blu Jam Cafe (323) 951-9191 7371 Melrose Ave, Los Angeles, CA


In [10]:
results_df = text_sim.match(queries, targets)

with pd.option_context('display.max_colwidth', None):
    display(results_df.head(10))

,query,target,similarity,is_match
0,"Saigon Sandwich (415) 474-5698 560 Larkin St, San Francisco, CA 94102","Saigon Sandwich (415) 474-5698 560 Larkin Street, San Francisco, CA",0.945792,True
1,"Sapp Coffee Shop (323) 665-1035 5183 Hollywood Blvd, Los Angeles, CA 90027","Hero Shop (213) 265-7561 130 E 6th St, Los Angeles, CA",0.750662,False
2,"Dong II Jang Restaurant (213) 383-5757 3455 W 8th St, Los Angeles, CA 90005","Dong Il Jang (213) 383-5757 3455 W 8th St, Los Angeles, CA",0.895420,False
3,"ZuZu Cafe & Market (608) 260-9898 1336 Drake St, Madison, WI 53715","Zuzu Cafe (608) 260-9898 1336 Drake St, Madison, WI",0.920317,True
4,"BJs Kolaches (715) 340-6267 975 University Ave, Madison, WI 53706","Subway (608) 441-6887 2850 University Ave, Madison, WI",0.733150,False
5,"Simone G.'s review of West River Caf (773) 866-2133 4400 N Kedzie Ave, Chicago, IL 60625","Vee Vee's (773) 465-2424 6232 N. Broadway Street, Chicago, IL",0.681165,False
6,"The Plaza Tavern (608) 255-6592 319 N Henry St, Madison, WI 53703","Plaza Tavern & Grill (608) 255-6592 319 N Henry St, Madison, WI",0.929599,True
7,"Kopi Kopi NYC (212) 777-7285 68 W 3rd St, New York, NY 10012","Stout Restaurant & Bar (212) 629-6191 133 W 33rd Street, New York, NY",0.706246,False
8,"Brian S.'s review of Hong Kong Loung (415) 668-8836 5322 Geary Blvd, San Francisco, CA 94121","Men Oh (415) 386-8802 5120 Geary Boulevard, San Francisco, CA",0.716806,False
9,"Bath & Body Works (608) 826-0626 23 W Towne Mall, Madison, WI 53719","Fast Wok (608) 833-6738 106 W Towne Mall, Madison, WI",0.784806,False


### Indexing and Searching for Similar Texts in a Dataset

TextSim allows you to maintain, update, and query a large index to find similar texts. This gives you more control over indexing and querying your dataset, including how many similar texts you want to retrieve per query and detailed results.

You can use the `.add` method to add examples from your dataset to the index, then use  `.search` to search the index and return the most similar texts to your search query.

In [11]:
# resets the index if you previously added things
text_sim.reset_index()

# adds the dataset of target examples to the index
text_sim.add(targets)

# for each query, search for the k=5 most similar texts
result_collection = text_sim.search(queries, similarity_threshold=0.9, k=5)

In [12]:
# texts are considered near-duplicate matches if their similarity >= similarity_threshold
total_matches = result_collection.total_matches
print("Total matches found:", total_matches)

Total matches found: 103


`result_collection.results` contains a list of results corresponding to each query. Each `Result` object contains the results of a search query, including the number of matches found (`.num_matches`), the idx/data/embedding of the query (`.query_idx`, `.query_data`, `.query_embedding`), and a list of `Match` objects (`.matches`).

The list of `Match`'s correspond to the `k` most similar texts found for the query, sorted by similarity (most similar first). Each `Match` object contains info on whether it is a near-duplicate match (`.is_match`), the rank (`.rank`), the data (`.data`), the similarity value (`.similarity`), and the embedding (`.embedding`) of the matched text.

You can visualize a search result using `text_sim.visualize(result)`.

In [13]:
# visualize results for each query using .visualize
query_idx = 0
result = result_collection.results[query_idx]
text_sim.visualize(result)

Query 0: "Saigon Sandwich (415) 474-5698 560 Larkin St, San Francisco, CA 94102"
Most similar matches:

  idx  is_match      similarity  text
-----  ----------  ------------  ----------------------------------------------------------------
  280  True                0.95  Saigon Sandwich (415) 474-5698 560 Larkin Street, San Francisco,
  106  False               0.72  Rin's (415) 821-4776 4301 24th Street, San Francisco, CA
   60  False               0.72  La Santaneca (415) 648-1034 3781 Mission Street, San Francisco,
  211  False               0.71  Pakwan (415) 255-2440 3182 16th Street, San Francisco, CA
  290  False               0.7   Rosamunde Sausage Grill (415) 437-6851 545 Haight Street, San Fr


In [14]:
# visualize a matching result
first_matching_idx = None
for result in result_collection.results:
    if result.num_matches > 0:
        first_matching_idx = result.query_idx
        break

result = result_collection.results[first_matching_idx]
text_sim.visualize(result)

Query 0: "Saigon Sandwich (415) 474-5698 560 Larkin St, San Francisco, CA 94102"
Most similar matches:

  idx  is_match      similarity  text
-----  ----------  ------------  ----------------------------------------------------------------
  280  True                0.95  Saigon Sandwich (415) 474-5698 560 Larkin Street, San Francisco,
  106  False               0.72  Rin's (415) 821-4776 4301 24th Street, San Francisco, CA
   60  False               0.72  La Santaneca (415) 648-1034 3781 Mission Street, San Francisco,
  211  False               0.71  Pakwan (415) 255-2440 3182 16th Street, San Francisco, CA
  290  False               0.7   Rosamunde Sausage Grill (415) 437-6851 545 Haight Street, San Fr


You can keep adding examples and querying your index after you create it. This is useful for production use-cases, where you have incoming data or frequently need to query your index.

In [15]:
# add new example to the index
new_examples = ["Googleplex (650) 253-0000 1600 Amphitheatre Parkway, Mountain View, CA 94043"]
text_sim.add(new_examples)

# search for the example, with typos in our query
result_collection = text_sim.search(["googleplx 650-253-0000 1600 amphitheatre parkway, mountain view, ca 94043"], k=5)

result = result_collection.results[0]
text_sim.visualize(result)

Query 0: "googleplx 650-253-0000 1600 amphitheatre parkway, mountain view, ca 94043"
Most similar matches:

  idx  is_match      similarity  text
-----  ----------  ------------  ----------------------------------------------------------------
  304  True                0.92  Googleplex (650) 253-0000 1600 Amphitheatre Parkway, Mountain Vi
  159  False               0.52  KFC (608) 849-5004 600 W Main St, Waunakee, WI
  158  False               0.51  Gus's Diner (608) 318-0900 630 N Westmount Dr, Sun Prairie, WI
  210  False               0.5   Sweet Maple (415) 655-9169 2101 Sutter Street, San Francisco, CA
   18  False               0.49  Pho Nam (608) 836-7040 610 Junction Rd Suite 109, Madison, WI
